In [16]:
import csv
import cv2
import numpy as np

In [11]:
"""Extract data from the .csv file"""
lines = []
with open('./data/driving_log.csv') as file:
    reader = csv.reader(file)
    for line in reader:
        lines.append(line)
 # images and mearsurments will be our input_X and input_y
images = []
mearsurements = []
for i in range(len(lines)):
    if i == 0:
        continue
    line = lines[i]
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    mearsurement = float(line[3])
    mearsurements.append(mearsurement)

In [17]:
"""Set X_train and y_train"""
X_train = np.array(images)
y_train = np.array(mearsurements)

In [22]:
"""Build a simple regression model"""
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=(160,320,3)))
# output regression layer
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=6)

model.save("./models/model.h5")

Train on 6428 samples, validate on 1608 samples
Epoch 1/6
 224/6428 [>.............................] - ETA: 3s - loss: 74968988.3369

/home/nqiao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


6428/6428 [==============================] - 2s 275us/step - loss: 5034098.1359 - val_loss: 4497.8690
Epoch 2/6
6428/6428 [==============================] - 2s 277us/step - loss: 3426.6828 - val_loss: 2630.9384
Epoch 3/6
6428/6428 [==============================] - 2s 277us/step - loss: 2753.1253 - val_loss: 2527.7907
Epoch 4/6
6428/6428 [==============================] - 2s 284us/step - loss: 2280.5834 - val_loss: 2439.1238
Epoch 5/6
6428/6428 [==============================] - 2s 279us/step - loss: 2594.5006 - val_loss: 2665.7679
Epoch 6/6
6428/6428 [==============================] - 2s 263us/step - loss: 2325.8360 - val_loss: 1912.3570
